# Orderbook Implementation - Student Template

## Overview
Build an optimal orderbook using:
- **Heaps** (heapq) - Track best bid/ask prices
- **Deques** (collections.deque) - FIFO queues at each price level
- **Dictionaries** - Fast O(1) order lookups

## Why Negative Prices for Bids?
Python's `heapq` only implements **min heaps** (smallest at top). For bids, we need a **max heap** (highest price first).

**Solution**: Store negative prices!
- Bid prices: $100, $95, $90
- Store as: -100, -95, -90
- Min heap puts -100 at top → negate back: -(-100) = $100 ✓

## Imports

In [ ]:
from collections import deque
import heapq
from enum import Enum
from typing import List, Tuple, Optional, Dict

## Enums for Order Side and Status

In [ ]:
class Side(Enum):
    BUY = "BUY"
    SELL = "SELL"

class OrderStatus(Enum):
    OPEN = "OPEN"
    PARTIALLY_FILLED = "PARTIALLY_FILLED"
    FILLED = "FILLED"
    CANCELLED = "CANCELLED"

## Order Class
Represents a single order with all its properties.

In [ ]:
class Order:
    """Represents a single order in the orderbook."""

    def __init__(self, order_id: str, side: Side, price: float, quantity: int, timestamp: int):
        self.order_id = order_id
        self.side = side
        self.price = price
        self.quantity = quantity
        self.filled_quantity = 0
        self.status = OrderStatus.OPEN
        self.timestamp = timestamp

    def remaining_quantity(self) -> int:
        """Returns the unfilled quantity of the order."""
        return self.quantity - self.filled_quantity

    def __repr__(self):
        return (f"Order(id={self.order_id}, side={self.side.value}, "
                f"price={self.price}, qty={self.quantity}, "
                f"filled={self.filled_quantity}, status={self.status.value})")

## OrderBook Class - Initialization
The data structures are already set up for you. Study them carefully!

In [ ]:
class OrderBook:
    """
    An optimal orderbook implementation using:
    - Heaps to track best bid/ask prices (O(log n) insert/delete)
    - Deques for FIFO queues at each price level (O(1) append/popleft)
    - Dictionaries for O(1) order lookups
    """

    def __init__(self):
        """Initialize the orderbook data structures."""
        # Price level management: price -> deque of orders at that price
        self.bids: Dict[float, deque] = {}  # Buy orders
        self.asks: Dict[float, deque] = {}  # Sell orders

        # Heaps to track best prices
        # For bids: use negative prices to simulate max heap (highest price first)
        # For asks: use positive prices for min heap (lowest price first)
        self.bid_heap: List[float] = []
        self.ask_heap: List[float] = []

        # Fast O(1) order lookup by order_id
        self.orders: Dict[str, Order] = {}

        # Timestamp counter for order sequencing (ensures FIFO at same price)
        self.timestamp = 0

## TODO 1: Submit Order

**Time Complexity**: O(log P) where P is the number of unique price levels

**Hints**:
1. Check if order_id already exists in self.orders
2. Increment self.timestamp and create new Order object
3. Add order to self.orders dictionary
4. Determine which side (self.bids or self.asks)
5. If price not in that side's dict, create new deque and push to heap
   - For bids: `heapq.heappush(self.bid_heap, -price)`  # negative!
   - For asks: `heapq.heappush(self.ask_heap, price)`
6. Append order to the deque at that price level

In [ ]:
def submit_order(self, order_id: str, side: Side, price: float, quantity: int) -> bool:
    """
    Submit a new order to the orderbook.

    Args:
        order_id: Unique identifier for the order
        side: Side.BUY or Side.SELL
        price: Limit price for the order
        quantity: Number of units to buy/sell

    Returns:
        True if order was successfully added, False if order_id already exists
    """
    # TODO: Implement this method
    pass

# Add method to OrderBook class
OrderBook.submit_order = submit_order

## TODO 2: Cancel Order

**Time Complexity**: O(M) where M is the number of orders at that price level

**Hints**:
1. Lookup order from self.orders by order_id
2. Check if order exists and can be cancelled (not FILLED or CANCELLED)
3. Set order.status = OrderStatus.CANCELLED
4. Get the price level queue from self.bids or self.asks
5. Use queue.remove(order) to remove from deque (this is O(M))
6. If queue is now empty, delete that price from the dict

In [ ]:
def cancel_order(self, order_id: str) -> bool:
    """
    Cancel an existing order.

    Args:
        order_id: ID of the order to cancel

    Returns:
        True if order was cancelled, False if order not found or already filled/cancelled
    """
    # TODO: Implement this method
    pass

# Add method to OrderBook class
OrderBook.cancel_order = cancel_order

## TODO 3: Get Order Status

**Time Complexity**: O(1)

**Hint**: Simple dictionary lookup - just return self.orders.get(order_id)

In [ ]:
def get_order_status(self, order_id: str) -> Optional[Order]:
    """
    Get the current status of an order.

    Args:
        order_id: ID of the order to query

    Returns:
        Order object if found, None otherwise
    """
    # TODO: Implement this method
    pass

# Add method to OrderBook class
OrderBook.get_order_status = get_order_status

## TODO 4: Modify Order

**Time Complexity**: O(M + log P)

**Hints**:
1. Get the existing order and verify it exists
2. Save the side and filled_quantity
3. Determine final_price (new_price if provided, else order.price)
4. Determine final_quantity (new_quantity if provided, else order.quantity)
5. Check that final_quantity >= filled_quantity
6. Cancel the order using self.cancel_order(order_id)
7. Submit new order with self.submit_order(order_id, side, final_price, final_quantity)
8. Restore filled_quantity if it was > 0

In [ ]:
def modify_order(self, order_id: str, new_price: Optional[float] = None,
                 new_quantity: Optional[int] = None) -> bool:
    """
    Modify an existing order (loses time priority).

    Args:
        order_id: ID of the order to modify
        new_price: New price (if changing price)
        new_quantity: New quantity (if changing quantity)

    Returns:
        True if modified successfully, False otherwise
    """
    # TODO: Implement this method
    pass

# Add method to OrderBook class
OrderBook.modify_order = modify_order

## TODO 5: Match Order

**Time Complexity**: O(log P + K×M + K×log P) where K = matched price levels, M = orders per level

**Hints**:
1. Submit the incoming order first
2. Determine opposite side:
   - If BUY: opposite is asks (match where ask_price <= buy_price)
   - If SELL: opposite is bids (match where bid_price >= sell_price)
3. Loop while incoming_order.remaining_quantity() > 0 and opposite_heap has prices:
   - Get best opposite price (remember to negate if using bid_heap)
   - Check if price is matchable (compare with incoming order's price)
   - Get the queue at that price level
   - Match against orders in FIFO order (while loop through queue)
   - Calculate trade_qty = min(incoming remaining, resting remaining)
   - Update filled_quantity for both orders
   - Update statuses (PARTIALLY_FILLED or FILLED)
   - Record trade (buyer_id, seller_id, trade_price, trade_qty)
   - Remove fully filled resting order from queue
   - Clean up empty price levels
4. Return list of trades

In [ ]:
def match_order(self, order_id: str, side: Side, price: float, quantity: int) -> List[Tuple[str, str, float, int]]:
    """
    Submit an order and match it against the opposite side of the book.

    Args:
        order_id: Unique identifier for the incoming order
        side: Side.BUY or Side.SELL
        price: Limit price for the order
        quantity: Number of units to buy/sell

    Returns:
        List of trades executed as (buyer_order_id, seller_order_id, trade_price, trade_quantity)
    """
    # TODO: Implement this method
    pass

# Add method to OrderBook class
OrderBook.match_order = match_order

## TODO 6: Get Best Bid

**Time Complexity**: Amortized O(1)

**Hints**:
1. Loop while self.bid_heap has items
2. Get price from heap top: `price = -self.bid_heap[0]` (negate!)
3. Check if price exists in self.bids and has orders
4. If valid, return price
5. Otherwise, pop stale price and clean up

In [ ]:
def get_best_bid(self) -> Optional[float]:
    """Get the best (highest) bid price."""
    # TODO: Implement using heap
    pass

# Add method to OrderBook class
OrderBook.get_best_bid = get_best_bid

## TODO 7: Get Best Ask

**Time Complexity**: Amortized O(1)

**Hints**: Same as get_best_bid but:
- Use self.ask_heap and self.asks
- Don't negate price: `price = self.ask_heap[0]` (already positive)

In [ ]:
def get_best_ask(self) -> Optional[float]:
    """Get the best (lowest) ask price."""
    # TODO: Implement using heap
    pass

# Add method to OrderBook class
OrderBook.get_best_ask = get_best_ask

## TODO 8: Get Spread

**Time Complexity**: Amortized O(1)

**Hint**: Call get_best_bid() and get_best_ask(), return their difference

In [ ]:
def get_spread(self) -> Optional[float]:
    """Get the bid-ask spread."""
    # TODO: Implement
    pass

# Add method to OrderBook class
OrderBook.get_spread = get_spread

## TODO 9: Pretty Print (Optional)

Display the orderbook in a readable format.

In [ ]:
def __repr__(self):
    """Pretty print the orderbook state."""
    lines = ["=" * 60]
    lines.append("ORDER BOOK")
    lines.append("=" * 60)

    # TODO: Implement pretty printing
    # Show asks (sorted high to low), then bids (sorted high to low)

    return "\n".join(lines)

# Add method to OrderBook class
OrderBook.__repr__ = __repr__

## Test Your Implementation

Run these cells to test your orderbook!

In [ ]:
# Test 1: Create orderbook and submit orders
ob = OrderBook()

print("Test 1: Submitting orders...")
ob.submit_order("B1", Side.BUY, 100.0, 50)
ob.submit_order("B2", Side.BUY, 99.5, 30)
ob.submit_order("A1", Side.SELL, 101.0, 40)
ob.submit_order("A2", Side.SELL, 101.5, 25)

print(f"Best Bid: {ob.get_best_bid()}")
print(f"Best Ask: {ob.get_best_ask()}")
print(f"Spread: {ob.get_spread()}")
print(ob)

In [ ]:
# Test 2: Check order status
print("Test 2: Checking order status...")
order = ob.get_order_status("B1")
print(f"Order B1: {order}")

In [ ]:
# Test 3: Cancel order
print("Test 3: Cancelling order B2...")
ob.cancel_order("B2")
print(f"Best Bid after cancel: {ob.get_best_bid()}")
print(ob)

In [ ]:
# Test 4: Modify order
print("Test 4: Modifying order...")
ob.submit_order("B3", Side.BUY, 99.0, 20)
ob.modify_order("B3", new_price=100.5)
print(ob)

In [ ]:
# Test 5: Match orders
print("Test 5: Matching a sell order at 99.5 for 60 units...")
trades = ob.match_order("S1", Side.SELL, 99.5, 60)
print(f"Trades executed: {len(trades)}")
for trade in trades:
    print(f"  Buyer: {trade[0]}, Seller: {trade[1]}, Price: {trade[2]}, Qty: {trade[3]}")
print(ob)

In [ ]:
# Test 6: Match across multiple price levels
print("Test 6: Matching a buy order at 102.0 for 100 units...")
trades = ob.match_order("B4", Side.BUY, 102.0, 100)
print(f"Trades executed: {len(trades)}")
for trade in trades:
    print(f"  Buyer: {trade[0]}, Seller: {trade[1]}, Price: {trade[2]}, Qty: {trade[3]}")
print(ob)